In [ ]:
import weather_data as wd
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import subprocess
import aiml
import os
import matplotlib.image as mpimg
import IPython
from IPython.display import IFrame

In [ ]:
def abbr_to_wind(abbr):
    if abbr == "VAR":
        return "geen"
    if 't' in abbr or 'd' in abbr:
        return abbr.capitalize()
    wind_direction = {'N': "noord", 'Z': "zuid", 'O': "oost", 'W': "west"}
    direction_string = ''
    for char in abbr:
        direction_string += wind_direction[char]
    return direction_string

def wind_to_abbr(wind):
    if wind == "geen":
        return "VAR"
    wind_dict = {"noord": 'N', "zuid": 'Z', "oost": 'O', "west": 'W'}
    for direction in wind_dict:
        wind = wind.replace(direction, wind_dict[direction])
    return wind
 
print(abbr_to_wind("NNO"))
print(wind_to_abbr("westoostzuid"))
    
def display_wind(winddir):
    return(mpimg.imread("images/"+wind_to_abbr(winddir)+".png"))

np.sum(display_wind("noordnoordoost"))

In [ ]:
# Create the kernel and learn AIML files
kernel = aiml.Kernel()
kernel.learn("start_bot.xml")
kernel.respond("load aiml")

kernel.setBotPredicate("name", "Sloeber")

# Press CTRL-C to break this loop
while True:
    message = input("Enter your message >> ")
    if message == "quit":
        break #useful in notebook
        # exit() # useful in actual application
    elif message == "save":
        kernel.saveBrain("aiml/bot_brain.brn")
    else:
        bot_response = kernel.respond(message)
        if kernel.getPredicate("graph_temp") == "1":
            tmax_0 = int(kernel.getPredicate("d0tmax"))
            tmax_1 = int(kernel.getPredicate("d1tmax"))
            tmax_2 = int(kernel.getPredicate("d2tmax"))
            tmin_0 = int(kernel.getPredicate("d0tmin"))
            tmin_1 = int(kernel.getPredicate("d1tmin"))
            tmin_2 = int(kernel.getPredicate("d2tmin"))
            tmax = [tmax_0, tmax_1, tmax_2]
            tmin = [tmin_0, tmin_1, tmin_2]
            days = ["vandaag", "morgen", "overmorgen"]
            l2 = plt.plot(days, tmin, color = "blue", label="minimum temperatuur")
            l1 = plt.plot(days, tmax, color = "orange", label="maximum temperatuur")
            plt.xlabel("Tijdstip")
            plt.ylabel("Temperatuur")
            plt.legend()
            plt.show()
            kernel.setPredicate("graph_temp", "0")
            print(kernel.respond('SPECIFICS'))
            
        elif kernel.getPredicate("graph_temp") == "2":
            fig = plt.figure()
            fig.set_figheight(15)
            fig.set_figwidth(15)
            for i in ["d0", "d1", "d2"]:
                ax = fig.add_subplot(1,3,int(i[1])+1)
                plt.xticks([])
                plt.yticks([])
                plt.title("Dag: "+i[1] +", Windkracht: "+ kernel.getPredicate(i+"windk"))
                plt.imshow(display_wind(kernel.getPredicate(i+"windr")))
            plt.show()
            kernel.setPredicate("graph_temp", "0")
            print(kernel.respond('SPECIFICS'))
            
        elif kernel.getPredicate("graph_temp") == "3":
            graph_avail_stations = [6391, 6275, 6249, 6260, 6235, 6370, 6377, 6350, 6323, 6280, 6330, 6279,
                                    6251, 6225, 6270, 6269, 6348, 6380, 6273, 6344, 6240, 6267, 6290, 6242,
                                    6310, 6375, 6319, 6340]
            station_id = wd.weather_data(kernel.getPredicate("cur_weather_loc"))['stationid']
            if station_id not in graph_avail_stations:
                station_id = min(graph_avail_stations, key=lambda x:abs(x-station_id))
            iframe = '//gadgets.buienradar.nl/gadget/forecastandstation/{}/'.format(station_id)
            IPython.display.display(IFrame(iframe, width=300, height=190))
            kernel.setPredicate("graph_temp", "0")
            print(kernel.respond('SPECIFICS'))
            
        elif kernel.getPredicate("graph_temp") == "4":
            loc = kernel.getPredicate("cur_weather_loc")
            lat_long = wd.lat_lon(loc)
            iframe = "https://gadgets.buienradar.nl/gadget/zoommap/?lat={}&lng={}&overname=2&zoom=11&naam={}&size=3&voor=1".format(lat_long[0], lat_long[1], loc)
            IPython.display.display(IFrame(iframe, width=550, height=512))
            kernel.setPredicate("graph_temp", "0")
            print(kernel.respond('SPECIFICS'))
        
        elif kernel.getPredicate("flag") == "2":
            t = int(round(float(kernel.getPredicate("gtemp"))))
            if t < 5:
                if float(kernel.getPredicate("d0neerslag")) < 30:
                    kernel.getPredicate("flag") == "0"
                    print(kernel.respond('COLD DRY'))
                else:
                    kernel.getPredicate("flag") == "0"
                    print(kernel.respond('COLD RAIN'))
            elif t < 15:
                if float(kernel.getPredicate("d0neerslag")) < 30:
                    kernel.getPredicate("flag") == "0"
                    print(kernel.respond('MIDDLE DRY'))
                else:
                    kernel.getPredicate("flag") == "0"
                    print(kernel.respond('MIDDLE RAIN'))
            else:
                if float(kernel.getPredicate("d0neerslag")) < 30:
                    kernel.getPredicate("flag") == "0"
                    print(kernel.respond('WARM DRY'))
                else:
                    kernel.getPredicate("flag") == "0"
                    print(kernel.respond('WARM RAIN'))
            kernel.setPredicate("flag", "0")
            print(kernel.respond('SPECIFICS'))
            
        elif kernel.getPredicate("flag") == "1":
            weather_dict = wd.weather_data(kernel.getPredicate("cur_weather_loc"))
            kernel.setPredicate("weather_descr", weather_dict["samenv"])
            kernel.setPredicate("temp", weather_dict["temp"])
            kernel.setPredicate("weather_forecast", weather_dict["verw"])
            print(kernel.respond('CURRENT WEATHER LOC'))
            print(kernel.respond('SPECIFICS'))
            for i in weather_dict:
                if "windr" in i: 
                    kernel.setPredicate(i, abbr_to_wind(weather_dict[i]))
                else:
                    kernel.setPredicate(i, weather_dict[i])
            kernel.setPredicate("flag", "0")
        else:
            print(bot_response)